In [1]:
import codecs  # Importe le module codecs pour travailler avec l'encodage des fichiers texte.
import nltk  # Importe le module nltk pour le traitement du langage naturel.
import random  # Importe le module random pour générer des nombres aléatoires.
import re  # Importe le module re pour travailler avec les expressions régulières.
import string  # Importe le module string pour les opérations sur les chaînes de caractères.
import csv  # Importe le module csv pour lire et écrire des fichiers CSV.
import inflect  # Importe le module inflect pour la conversion de nombres en mots.
from count_syllables import count_syllables  # Importe la fonction count_syllables du module count_syllables.

In [7]:
class PoemGenerator(object):
    def __init__(self, corpus='buzzfeed_facebook_statues.csv'):
        # Initialise la classe PoemGenerator avec un corpus par défaut.
        self.only_punctuation = re.compile(r'[^\w\s]+$')
        # Compile une expression régulière pour trouver la ponctuation uniquement.
        self.spaces_and_punctuation = re.compile(r"[\w']+|[.,!?;]")
        # Compile une expression régulière pour diviser les mots et la ponctuation.
        self.sents = []
        # Initialise une liste pour stocker les phrases du corpus.
        self.words = []
        # Initialise une liste pour stocker les mots du corpus.
        self.all_words = []
        # Initialise une liste pour stocker tous les mots (sans ponctuation) du corpus.
        self.inflect_engine = inflect.engine()
        # Initialise le moteur d'inflection pour la conversion de nombres en mots.
        self.read_corpus(corpus)
        # Appelle la méthode read_corpus pour charger le corpus spécifié.
        self.bigrams = list(nltk.bigrams(self.words))
        # Crée une liste de bigrammes à partir des mots du corpus.
        self.cfd = nltk.ConditionalFreqDist(self.bigrams)
        # Initialise une fréquence conditionnelle des bigrammes.
        self.history = []
        # Initialise une liste pour suivre l'historique des générations de poèmes.

    def read_corpus(self, corpus):
        """Given filename of corpus, populate words, all_words, and sents."""
        if corpus.endswith('.csv'):
            if 'buzzfeed_facebook_statuses' in corpus:
                return self.read_buzzfeed_corpus(corpus)
            else:
                return self.read_csv_corpus(corpus)
        elif corpus.endswith('.txt'):
            return self.read_txt_corpus(corpus)
        else:
            raise TypeError(('Unrecognized corpus file type: %s.' % corpus) +
                            '".txt" and ".csv" are only supported')
        # Méthode pour lire le corpus en fonction du type de fichier spécifié.

    def read_txt_corpus(self, corpus):
        with codecs.open(corpus, 'r', 'utf-8') as corpus_content:
            text = corpus_content.read()
            sents = nltk.tokenize.sent_tokenize(text)
            words = nltk.tokenize.word_tokenize(text)
            self.sents.extend(sents)
            self.words.extend(words)
            self.all_words.extend([word for word in words
                                   if not
                                   self.only_punctuation.match(word)])
        # Méthode pour lire un fichier texte, diviser en phrases et mots, et les ajouter aux listes appropriées.

    def read_buzzfeed_corpus(self, corpus):
        with open(corpus, newline='', encoding='utf-8') as statuses:
            reader = csv.reader(statuses, delimiter=',')
            for row in reader:
                if 'via buzzfeed ' not in row[1].lower():  # only English
                    # split title into a list of words and punctuation
                    title = self.spaces_and_punctuation.findall(row[2])
                    # spell out digits into ordinal words for syllable counting
                    title = [string.capwords(
                             self.inflect_engine.number_to_words(int(word)))
                             if word.isdigit() else word for word in title]
                    self.sents.append(title)
                    self.words.extend(title)
                    self.all_words.extend([word for word in title
                                           if not
                                           self.only_punctuation.match(word)])
        # Méthode pour lire un fichier CSV spécifique à BuzzFeed, extraire les titres, et les ajouter aux listes appropriées.

    def markov(self, word, n):
        if n > 0:
            print(word,)
            n = n - 1
            self.markov(random.choice(self.cfd[word].items())[0], n)
        else:
            print('')
        # Méthode pour générer du texte à l'aide d'une chaîne de Markov avec une profondeur spécifiée.

    def generate_text(self):
        word = random.choice(self.bigrams)[0]
        self.markov(word, 15)
        # Méthode pour générer du texte en utilisant la méthode markov avec une longueur spécifiée.

    def haiku_line(self, line, current_syllables, next_words, target_syllables):
        if next_words == []:
            # this branch failed
            return None
        else:
            word = random.choice(next_words)
        new_line = line[:]
        new_line.append(word)
        new_syllables = sum(map(count_syllables, new_line))
        if new_syllables == target_syllables:
            return new_line
        elif new_syllables > target_syllables:
            new_next_words = next_words[:]
            new_next_words.remove(word)
            return self.haiku_line(line, current_syllables, new_next_words, target_syllables)
        else:
            new_next_words = [freq[0] for freq in self.cfd[word].items()
                              if not self.only_punctuation.match(freq[0])]
            branch = self.haiku_line(new_line, new_syllables, new_next_words, target_syllables)
            if branch:
                return branch
            else:
                new_next_words = next_words[:]
                new_next_words.remove(word)
                return self.haiku_line(line, current_syllables, new_next_words, target_syllables)
        # Méthode pour générer une ligne de haïku en utilisant la récursivité pour atteindre le nombre de syllabes cible.

    def generate_haiku(self):
        haiku = ''
        first = self.haiku_line([], 0, self.all_words, 5)
        haiku = haiku + ' '.join(first) + '\n'
        next_words = [freq[0] for freq in self.cfd[first[-1]].items()
                      if not self.only_punctuation.match(freq[0])]
        if not next_words:
            next_words = self.all_words
        second = self.haiku_line([], 0, next_words, 7)
        haiku = haiku + ' '.join(second) + '\n'
        next_words = [freq[0] for freq in self.cfd[second[-1]].items()
                      if not self.only_punctuation.match(freq[0])]
        if not next_words:
            next_words = self.all_words
        third = self.haiku_line([], 0, next_words, 5)
        haiku = haiku + ' '.join(third) + '\n'
        return haiku
    # Méthode pour générer un haïku en appelant la méthode haiku_line trois fois pour les trois lignes.

    def generate_endless_poem(self, previous_line):
        random_syllables = random.choice(range(1, 26))
        if previous_line is None:
            next = self.haiku_line([], 0, self.all_words, random_syllables)
            print(' '.join(next))
        else:
            next_words = [freq[0] for freq in self.cfd[previous_line[-1]].items()
                          if not self.only_punctuation.match(freq[0])]
            next = self.haiku_line([], 0, next_words, random_syllables)
            print(' '.join(next))
        self.generate_endless_poem(next)
    # Méthode pour générer un poème sans fin en utilisant la méthode haiku_line avec une ligne précédente comme point de départ.

    def generate_sonnet(self):
        sonnet = ""
        # Quatrains (4 quatrains)
        for _ in range(4):
            line1 = self.haiku_line([], 0, self.all_words, 10)  # 10 syllables per line
            line2 = self.haiku_line([], 0, self.all_words, 10)
            line3 = self.haiku_line([], 0, self.all_words, 10)
            line4 = self.haiku_line([], 0, self.all_words, 10)
            sonnet += ' '.join(line1) + '\n'
            sonnet += ' '.join(line2) + '\n'
            sonnet += ' '.join(line3) + '\n'
            sonnet += ' '.join(line4) + '\n'

        # Distiche (2 vers)
        line5 = self.haiku_line([], 0, self.all_words, 10)
        line6 = self.haiku_line([], 0, self.all_words, 10)
        sonnet += ' '.join(line5) + '\n'
        sonnet += ' '.join(line6) + '\n'

        return sonnet
    # Méthode pour générer un sonnet en utilisant la méthode haiku_line pour chaque ligne du sonnet.


In [11]:
if __name__ == '__main__':
    generator = PoemGenerator(corpus='buzzfeed_facebook_statuses.csv')
    haiku = generator.generate_haiku()
    sonnet = generator.generate_sonnet()
    print(haiku)
    print('=' * 30, '\n')
    print(sonnet)

King Is Coming Back
With Spaghetti And Kourtney
Kardashian Used Art


By Surprising Dance Moves What Teens Say It
She Went For Greatest Movies Told The Week
Around Your Drink Of Social Media
In Ohio Man Arrested And Jay
The Prisoners The Miley Cyrus' VMA
Chris Kelly Ignition Remix Is Six
Renaissance Baby We've Tasted Them When
Photos McDonald's Was Headed Towards
Sixteen Eyebrow Diagrams To Visit
Could Hope From Mad A Pic A Fox News Blurred
Baby Cub Scare Prank How Old Dancers in
You McDonald's Or Alan Gross Photos
Phone In Mary Kate Ashley Have Starred On
Under 5'4 Twenty-nine Puntastic Jokes
Dyed Hair Will Perform Salt N Pepa And
The Giant Camel Crucial Booze This Dog
Shop Seventeen Razones por frases de
Every Bra Seventeen Tips From MTV



In [10]:
endless_poem = generator.generate_endless_poem(None)
print(endless_poem)

New Miss America You Chris
Pratt's
Is Arrested Is Blue Dress Was Downed Malaysia Airlines Sends Her
Husband's Suspension From
Spending Ten Lovely Version Of Lana Del Rey's New
Pregnancy And Gay This Incredibly Touching
Photos Guaranteed To Listen To Wait
Archie Picks Betty PIC Extreme Oreo Got One Hundred And Forty-two Hugs That Seem
What's At Swimming Naked Together Can Pull This
Ink Blot Test In Forty-seven Brides For Oh My Body Twenty-seven Heart Twenty-three
Completely Geek Out Twenty-three Gadgets That Never Make His Oscars Performance These Cult Films
This Message To Join Augusta National High With Secretly Attracts You Happy The
Thirty Look at Play F k Is Running Out Grumpy Cat Why This Zombie Apocalypse College
Major Announcement Photo Ninety-four Reasons Not What Fictional City
Chiefs The RNC Campbell's Without Your Imaginary World Twenty-eight Pictures One Thing You've Already
Testing Its Time Taylor Swift's Fight Ebola
Had Names Say She's
Homeless Eleven Struggles Everyone W

TypeError: can only join an iterable